# HW6 - Transforming vote tallies

This homework give you some insight into the transformations that are underlying all the voting dashboards you've undoubtedly seen all over the news.  Here we are going to use 2020 Presidential election data.

The dataset for this week contains voting outcomes for different batches of votes in the battleground states.  The data consist of the following:
* The state of the votes
* The time the voting results were reported
* The number of votes in the batch (new_votes)
* The number of those new votes that were for Joe Biden (votes_biden)

The key metric of interest is in how each candidate, Biden and Trump, are doing over time in terms of the percentage of votes they're getting with each batch.  This is easier to think of compared to raw numbers for a variety of reasons.  One is that the number of votes varies state-to-state and batch-to-batch, so just thinking about raw numbers doesn't inform very much.  The other is that people are often thinking about the percentage a candidate needs to take the lead or win the state.  If each batch is hitting at or above that percentage, then they have a good chance of winning the state.  

The goal of this homework is to generate the percentages of votes in each state that are going to Trump and Biden **on an hourly reporting basis**. Each line is a batch of votes that were reported at a given time.  Sometimes there are multiple in an hour.  We want to report a coarser resolution which is why we want to group by hours and days.  

Given you have only the variables listed above, you're going to need to do a few things to make this happen:
* Import your data and apply a timestamp
* Make a column for the number of votes received by Trump
* Aggregate your data to get the number of votes for each on a daily and hourly basis
* Create columns of the percentages of votes each have received in the aggregated data

**NOTE 1** - This is by every measure 'small' data, but it needs to be coded in pyspark.

**NOTE 2** - These are all votes that were reported after the initial waves of in-person and early reported mail-in votes.  So if you explore you might see that the total votes in this dataset don't seem to match up with what you would find elsewhere.  That's only because those early votes aren't added in.  These are just batch totals of later reported batches.



**Submission Instruction:**

* Make a copy and replace blank in the title with your name
* Run all the cells
* Download the notebook (.ipynb)
* Submit on Gradescope



In [ ]:
!pip install pyspark

## Importing your data - 3 point

The URL to import your data is

http://131.193.32.85:9000/mybucket/votes_2020_hw5.txt

You need to import the data with the appropriate options applied.  You will need to apply a timestampFormat option as well, but it's best to import the data first, look at the format, and then create your timestamp string.  Note, this is fractional seconds in the timestamp. <a href="https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html" target="_blank">Remember you can see a full list of datetime formatting here</a>

Also, although it's a text file you can import it as a CSV and specify a delimiter as an option.  

Call the imported dataframe 'votes'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("intro_pyspark") \
    .getOrCreate()

from pyspark.sql.functions import *

from pyspark import SparkFiles
url = 'http://131.193.32.85:9000/mybucket/votes_2020_hw5.txt'
spark.sparkContext.addFile(url)
# Make a filepath
fp = 'file://'+SparkFiles.get("votes_2020_hw5.txt")
fp

'file:///tmp/spark-2224f4ab-e09b-4b9d-b845-5db09f598cf5/userFiles-b5638f90-0358-473d-9740-4382e68d1844/votes_2020_hw5.txt'

In [3]:
# import as votes
votes = spark.read.option('header', True).csv(fp)

In [4]:
# Check data
votes.show()

+----------------+--------------------+---------+-----------+
|           state|           timestamp|new_votes|votes_biden|
+----------------+--------------------+---------+-----------+
|  Alaska (EV: 3)|2020-11-09 19:14:...|    18939|     7779.0|
|  Alaska (EV: 3)|2020-11-04 18:40:...|    39556|    11448.0|
|  Alaska (EV: 3)|2020-11-04 13:28:...|        0|        0.0|
|Arizona (EV: 11)|2020-11-10 02:18:...|     6397|     3127.0|
|Arizona (EV: 11)|2020-11-10 00:51:...|      600|      232.0|
|Arizona (EV: 11)|2020-11-10 00:02:...|      591|       84.0|
|Arizona (EV: 11)|2020-11-09 22:23:...|     3437|     1049.0|
|Arizona (EV: 11)|2020-11-09 21:15:...|      787|      189.0|
|Arizona (EV: 11)|2020-11-09 18:40:...|      424|      250.0|
|Arizona (EV: 11)|2020-11-09 18:30:...|      145|      114.0|
|Arizona (EV: 11)|2020-11-09 18:16:...|      124|       57.0|
|Arizona (EV: 11)|2020-11-09 00:49:...|      327|      180.0|
|Arizona (EV: 11)|2020-11-08 23:35:...|    16739|     6748.0|
|Arizona

## Create number of votes for trump - 3 points

Just like it sounds.  Create a column in votes called 'votes_trump' that has the number of votes he received in the batch.

You'll need to import all the functions from pyspark.sql.functions to do this and later steps.

In [5]:
from pyspark.sql.functions import *

# make votes_trump
votes = votes.withColumn('votes_trump', col('new_votes') - col('votes_biden'))

In [6]:
# Check
votes.show()

+----------------+--------------------+---------+-----------+-----------+
|           state|           timestamp|new_votes|votes_biden|votes_trump|
+----------------+--------------------+---------+-----------+-----------+
|  Alaska (EV: 3)|2020-11-09 19:14:...|    18939|     7779.0|    11160.0|
|  Alaska (EV: 3)|2020-11-04 18:40:...|    39556|    11448.0|    28108.0|
|  Alaska (EV: 3)|2020-11-04 13:28:...|        0|        0.0|        0.0|
|Arizona (EV: 11)|2020-11-10 02:18:...|     6397|     3127.0|     3270.0|
|Arizona (EV: 11)|2020-11-10 00:51:...|      600|      232.0|      368.0|
|Arizona (EV: 11)|2020-11-10 00:02:...|      591|       84.0|      507.0|
|Arizona (EV: 11)|2020-11-09 22:23:...|     3437|     1049.0|     2388.0|
|Arizona (EV: 11)|2020-11-09 21:15:...|      787|      189.0|      598.0|
|Arizona (EV: 11)|2020-11-09 18:40:...|      424|      250.0|      174.0|
|Arizona (EV: 11)|2020-11-09 18:30:...|      145|      114.0|       31.0|
|Arizona (EV: 11)|2020-11-09 18:16:...

## Aggregate data - 6 points

Time to group your data. You're going to need to group by day, hour, and then state to get the totals.  I showed you how to group by a single column that's not a datetime in the previous lesson.  Do do more than one you just add them in separated by commas.  The only issue is that you need to extract day and hour from the timestamp.  To do this for the day of the month your grouping variable would be `dayofmonth('timestamp')`.  I'll let you figure out how to do it for hour and day.  Grouping by state should be easy.

For the aggregation you need to calculate the sums that will allow you to get the percentage of votes that went to Biden vs Trump in a given hour.

Call the resulting dataframe votes_hourly

In [9]:
# make votes_hourly
votes_hourly = votes.groupBy(
    dayofmonth('timestamp').alias('day'),
    hour('timestamp').alias('hour'),
    'state'
).agg(
    sum('new_votes').alias('total_new_votes'),
    sum('votes_biden').alias('total_votes_biden'),
    sum('votes_trump').alias('total_votes_trump')
).orderBy('day', 'hour', 'state')

votes_hourly.show()


+---+----+--------------------+---------------+-----------------+-----------------+
|day|hour|               state|total_new_votes|total_votes_biden|total_votes_trump|
+---+----+--------------------+---------------+-----------------+-----------------+
|  4|  13|      Alaska (EV: 3)|            0.0|              0.0|              0.0|
|  4|  13|    Arizona (EV: 11)|            0.0|              0.0|              0.0|
|  4|  13|    Georgia (EV: 16)|            0.0|              0.0|              0.0|
|  4|  13|      Nevada (EV: 6)|            0.0|              0.0|              0.0|
|  4|  13|North Carolina (E...|            0.0|              0.0|              0.0|
|  4|  13|Pennsylvania (EV:...|            0.0|              0.0|              0.0|
|  4|  14|    Arizona (EV: 11)|       126904.0|          44544.0|          82360.0|
|  4|  14|    Georgia (EV: 16)|           78.0|             67.0|             11.0|
|  4|  14|Pennsylvania (EV:...|        44229.0|          36961.0|           

## Calculating percentage of votes per hour - 6 points

Now go and make your columns of the percentage of votes received each hour by each candidate. Call these 'percent_biden' and 'percent_trump'

In [10]:
votes_hourly = votes_hourly.withColumn(
    'percent_biden',
    (col('total_votes_biden') / col('total_new_votes')) * 100
).withColumn(
    'percent_trump',
    (col('total_votes_trump') / col('total_new_votes')) * 100
)

votes_hourly.show()

+---+----+--------------------+---------------+-----------------+-----------------+-----------------+------------------+
|day|hour|               state|total_new_votes|total_votes_biden|total_votes_trump|    percent_biden|     percent_trump|
+---+----+--------------------+---------------+-----------------+-----------------+-----------------+------------------+
|  4|  13|      Alaska (EV: 3)|            0.0|              0.0|              0.0|             NULL|              NULL|
|  4|  13|    Arizona (EV: 11)|            0.0|              0.0|              0.0|             NULL|              NULL|
|  4|  13|    Georgia (EV: 16)|            0.0|              0.0|              0.0|             NULL|              NULL|
|  4|  13|      Nevada (EV: 6)|            0.0|              0.0|              0.0|             NULL|              NULL|
|  4|  13|North Carolina (E...|            0.0|              0.0|              0.0|             NULL|              NULL|
|  4|  13|Pennsylvania (EV:...| 

## Rounding values - 3 point

Those percent_biden and percent_trump columns have too many digits.  You can import a rounding `round()` function from pyspark using `from pyspark.sql.functions import round`.  Go and import that function and then apply round to those columns so that they have 3 digits after the decimal.  It's fine to overwrite them with the rounded values.

In [11]:
votes_hourly = votes_hourly.withColumn(
    'percent_biden',
    round(col('percent_biden'), 3)
).withColumn(
    'percent_trump',
    round(col('percent_trump'), 3)
)

votes_hourly.show()


+---+----+--------------------+---------------+-----------------+-----------------+-------------+-------------+
|day|hour|               state|total_new_votes|total_votes_biden|total_votes_trump|percent_biden|percent_trump|
+---+----+--------------------+---------------+-----------------+-----------------+-------------+-------------+
|  4|  13|      Alaska (EV: 3)|            0.0|              0.0|              0.0|         NULL|         NULL|
|  4|  13|    Arizona (EV: 11)|            0.0|              0.0|              0.0|         NULL|         NULL|
|  4|  13|    Georgia (EV: 16)|            0.0|              0.0|              0.0|         NULL|         NULL|
|  4|  13|      Nevada (EV: 6)|            0.0|              0.0|              0.0|         NULL|         NULL|
|  4|  13|North Carolina (E...|            0.0|              0.0|              0.0|         NULL|         NULL|
|  4|  13|Pennsylvania (EV:...|            0.0|              0.0|              0.0|         NULL|       

## SQL query - 9 points

Now let's leverage those SQL powers to summarize our data a bit more within Georgia, which was a critical state for 2020 election.  I want you to write two queries.  
1. The first should be just getting all data from Georgia. You can use "Like '%Georgia%' " to match for Georigia state in your query (you may see negative values because the data has not been cleaned).
2. The second should get the daily percent of votes going to Trump for each state. Keep in mind that the percent votes will need to be recalculated (ie averaging the hourly percentages won't work).

Remember you need to register your dataframe as a table to access it!

In [12]:
# query 1
votes_hourly.createOrReplaceTempView("votes_hourly_table")

query1 = """
SELECT *
FROM votes_hourly_table
WHERE state LIKE '%Georgia%'
ORDER BY day, hour
"""

georgia_votes = spark.sql(query1)
georgia_votes.show()

+---+----+----------------+---------------+-----------------+-----------------+-------------+-------------+
|day|hour|           state|total_new_votes|total_votes_biden|total_votes_trump|percent_biden|percent_trump|
+---+----+----------------+---------------+-----------------+-----------------+-------------+-------------+
|  4|  13|Georgia (EV: 16)|            0.0|              0.0|              0.0|         NULL|         NULL|
|  4|  14|Georgia (EV: 16)|           78.0|             67.0|             11.0|       85.897|       14.103|
|  4|  16|Georgia (EV: 16)|         3792.0|           2839.0|            953.0|       74.868|       25.132|
|  4|  17|Georgia (EV: 16)|        24305.0|          20241.0|           4064.0|       83.279|       16.721|
|  4|  18|Georgia (EV: 16)|         4131.0|           3516.0|            615.0|       85.113|       14.887|
|  4|  19|Georgia (EV: 16)|         7605.0|           5572.0|           2033.0|       73.268|       26.732|
|  4|  20|Georgia (EV: 16)| 

In [13]:
# query 2
query2 = """
SELECT
    day,
    state,
    ROUND((SUM(total_votes_trump) / SUM(total_new_votes)) * 100, 3) AS percent_trump_daily
FROM votes_hourly_table
GROUP BY day, state
ORDER BY day, state
"""

daily_trump_percent = spark.sql(query2)
daily_trump_percent.show()

+---+--------------------+-------------------+
|day|               state|percent_trump_daily|
+---+--------------------+-------------------+
|  4|      Alaska (EV: 3)|             71.059|
|  4|    Arizona (EV: 11)|              64.99|
|  4|    Georgia (EV: 16)|             28.457|
|  4|      Nevada (EV: 6)|               NULL|
|  4|North Carolina (E...|              100.0|
|  4|Pennsylvania (EV:...|             25.782|
|  5|    Arizona (EV: 11)|             58.879|
|  5|    Georgia (EV: 16)|              25.51|
|  5|      Nevada (EV: 6)|             43.187|
|  5|North Carolina (E...|              100.0|
|  5|Pennsylvania (EV:...|             26.827|
|  6|    Arizona (EV: 11)|             56.068|
|  6|    Georgia (EV: 16)|             31.658|
|  6|      Nevada (EV: 6)|             37.769|
|  6|North Carolina (E...|             42.979|
|  6|Pennsylvania (EV:...|             23.749|
|  7|    Arizona (EV: 11)|             59.016|
|  7|    Georgia (EV: 16)|             27.489|
|  7|      Ne